In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
import torch
from model.gpt import GPT


/home/palaash/miniconda3/envs/tinylm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# i = torch.ones(1,10).long().to(1)

In [9]:
model = GPT.from_hf('microsoft/Phi-3-mini-4k-instruct', enable_qlora=True) 

Unused kwargs: ['load_4bit_use_double_quant']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.44s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
module = model.layers

In [11]:
module._hf_hook.__class__

accelerate.hooks.AlignDevicesHook

In [35]:
from torch import nn
from accelerate.hooks import AlignDevicesHook
def remove_hook_from_module(module: nn.Module, recurse=False):
    """
    Removes any hook attached to a module via `add_hook_to_module`.

    Args:
        module (`torch.nn.Module`): The module to attach a hook to.
        recurse (`bool`, **optional**): Whether to remove the hooks recursively

    Returns:
        `torch.nn.Module`: The same module, with the hook detached (the module is modified in place, so the result can
        be discarded).
    """

    if hasattr(module, "_hf_hook"):
        module._hf_hook.detach_hook(module)
        # print(hasattr(module, "_hf_hook"))
        delattr(module, "_hf_hook")

    if hasattr(module, "_old_forward"):
        # Overriding a GraphModuleImpl forward freezes the forward call and later modifications on the graph will fail.
        # Reference: https://pytorch.slack.com/archives/C3PDTEV8E/p1705929610405409
        if "GraphModuleImpl" in str(type(module)):
            module.__class__.forward = module._old_forward
        else:
            module.forward = module._old_forward
            print('eca')
        # delattr(module, "_old_forward")
        module._old_forward = None

    if recurse:
        for child in module.children():
            remove_hook_from_module(child, recurse)

    return module

In [37]:
module._old_forward

<bound method Phi3ForCausalLM.forward of Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=3072, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=3072, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (qkv_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=3072, out_features=9216, 

In [36]:
module._hf_hook = AlignDevicesHook()
remove_hook_from_module(module)

eca


AttributeError: _old_forward

In [21]:
module._hf_hook = None

In [19]:
for cls in type(module).__mro__:  # Check all classes in the method resolution order
    if '_hf_hook' in cls.__dict__:
        print(f"Found _hf_hook in {cls.__name__}, {cls.__dict__['_hf_hook']}")

In [20]:
if '__getattr__' in dir(module) or '__getattribute__' in dir(module):
    print("Dynamic attribute handling is present.")


Dynamic attribute handling is present.


: 